In [1]:
# Load package
import numpy as np; from scipy import stats; import matplotlib.pyplot as plt; import pymc as pm;import arviz as az; 
import math; import pandas as pd
from scipy.optimize import minimize
from scipy.optimize import root
from scipy import special
import pytensor.tensor as pt
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.optimize import fsolve

import warnings
warnings.filterwarnings("ignore")

In [2]:
def generate_Corr_identity(p):

    Sigma = np.zeros((p-1, p-1))
    np.fill_diagonal(Sigma, 1)
    return Sigma

def generate_data(n, p, sigma_sqr, beta, nu, corr):

    beta = beta.reshape((p, 1))
    x_i = np.random.normal(0, 1, (n, p - 1))
    x_i_correlated = x_i @ corr
    ones = np.ones((n, 1))
    x_i_full =  np.concatenate((ones, x_i_correlated), axis=1)
    XB = x_i_full @ beta
    E = stats.t.rvs(df = nu, loc=0, scale= np.sqrt(sigma_sqr), size=(n, 1))
    Y = XB + E
    return Y, x_i_full,x_i

def calculate_y_axix(nu_origin, nu_est):
    n = len(nu_est)
    if n == 0:
        return -1
    else:
        nu_origin_vec = nu_origin * np.ones((n, 1))
        mse = np.sum((nu_est - nu_origin_vec)**2) / n
        result = np.sqrt(mse)/nu_origin
        return result
    
# full likelihood
def negative_log_likelihood(params):
    betas, sigma, nu = params[:-2], params[-2], params[-1]
    p = X.shape[1]
    B = np.reshape(betas, (p, 1))
    XB = X @ B
    n = X.shape[0]
    XB = XB.reshape((n, 1))
    
    # loglikelihood
    equation = n * np.log(special.gamma((nu + 1)/2)) + n* nu *0.5 * np.log(nu) - n * np.log(special.gamma(nu/2)) - 0.5*n*np.log(np.pi) - n * np.log(sigma) - 0.5 *(nu + 1)*np.sum(np.log(nu + ((y - XB)/sigma)**2))
    return -equation

# Jeffrey's prior
def logJeff(x):
    return np.log((x/(x+3))**(1/2)*(special.polygamma(1,x/2) - special.polygamma(1, (x+1)/2) - 2*(x+3)/(x*(x+1)**2))**(1/2))

# full joint
def negative_joint(params):
    nu = params[-1]
    return -logJeff(nu) + negative_log_likelihood(params)

def initial_guess_from_lin_reg(x_without_1, y,nu_origin):
    initial_guess = []
    
    model = LinearRegression().fit(x_without_1, y)
    # intercept 
    initial_guess.append(float(model.intercept_))
    # coeff
    for coeff in model.coef_[0]:
        initial_guess.append(coeff)
    # sigma_sq    
    y_pred = model.predict(x_without_1)
    residual_sq = (((y - y_pred)**2).sum())/(n-2)
    initial_guess.append(residual_sq)
    
    # nu
    initial_guess.append(nu_origin) # use true nu for initial guess
    return initial_guess

def optimizer_all_three_params_least_sq(eqt, initial_guess,method_name):
    p = X.shape[1]
    bounds = [(None, None)] * p +[(0, np.inf)]*2
    result = minimize(eqt, initial_guess, method= method_name,bounds = bounds, options={'maxiter':1000})
    return result

def fix_x_generate_data(n, p, sigma_sqr, beta, nu, corr,X):

    beta = beta.reshape((p, 1))
    XB = X @ beta
    E = stats.t.rvs(df = nu, loc=0, scale= np.sqrt(sigma_sqr), size=(n, 1))
    Y = XB + E
    return Y

In [3]:
def standard_error(nu):
    
    p = X.shape[1]
    B = np.reshape(beta, (p, 1))
    XB = X @ B
    n = X.shape[0]
    XB = XB.reshape((n, 1))
    Z = XB/sigma_sqr
    
    # special.polygamma(1,x) is trigamma at x 
    second_derivative_of_nu = n/4 * (special.polygamma(1, (nu+1)/2) - special.polygamma(1, nu/2)) + 1/2 * np.sum((Z**2)/(nu + (Z**2)) - (Z**2 -1)/(nu + (Z**2))**2)
    return 1/ np.sqrt(second_derivative_of_nu) 

def confidence_interval(MAP, sd):
    upper_interval = MAP + 1.96*sd
    lower_interval = MAP - 1.96*sd
    return [lower_interval,upper_interval]

In [ ]:
# Fix number of observations
n = 50
p = 5
beta = np.array([2, 1, 0.3, 0.9, 1])
sigma_sqr = 1.5
corr = generate_Corr_identity(p)
x_without_1 = np.random.normal(0, 1, (n, p - 1))
x_i_correlated = x_without_1 @ corr
ones = np.ones((n, 1))
X =  np.concatenate((ones, x_i_correlated), axis=1)

final_dict = {}
for nu_origin in range(5,21):
    CI_list = []
    MAP_list = []
    count_include_in_CI = 0
    
    for j in range(100): # number of simulations 
        y = fix_x_generate_data(n, p, sigma_sqr, beta, nu_origin, corr, X) 
        
        # Maximize log joint with NM algorithm and LS intial guess to get MAP
        initial_guess = initial_guess_from_lin_reg(x_without_1, y,nu_origin)
        profile_lse_joint_result = optimizer_all_three_params_least_sq(negative_joint, initial_guess, 'Nelder-Mead')
        if profile_lse_joint_result.success == True:
            MAP = profile_lse_joint_result.x[-1]
            MAP_list.append(MAP)
            sd = standard_error(MAP)
            CI_list.append(confidence_interval(MAP, sd))
            
    for CI in CI_list:
        count_include_in_CI += CI[0]<= nu_origin<= CI[1]
    MSE = calculate_y_axix(nu_origin, MAP_list)
    final_dict[nu_origin] = [count_include_in_CI, MSE]

In [ ]:
final_dict